In [1]:
from sk_embed_generation import EmbeddingGenerator
from sklearn.metrics.pairwise import euclidean_distances
from sk_model import SkipGramModel
import sk_embed_generation
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import itertools 
import pandas as pd

In [2]:
stem_data = EmbeddingGenerator('/Users/rsciagli/documents/Fall2020/BAR/STU_CRS_TBL_full.csv',
                              '/Users/rsciagli/documents/Fall2020/BAR/student_attribute_table_full.xlsx')

,COHORT_SP_FA,COHORT_TERM_CD,COHORT_TERM_DESC,ENTRY_ACADEMIC_LOAD,ENTRY_AGE,ENTRY_GPA,ENTRY_SCHOOL,ENTRY_INTENDED_SCHOOL,ENTRY_PROGRAM,ENTRY_MAJOR_CODE,...,TWNTYFRST_CENT_SCHOLAR,CLASS_LEVEL,ENTRY_INTENDED_SCHOOL1,ADMIT_STATUS,CERT_TERM_CD,TIME_2_CERT,ethnicity_id,urm_flag_id,gender_id,pell_eligibility_id
0,Fall,4068,Fall 2006,F,31,3.925,SCS,SCS,SCS1,GENSTBGS,...,N,Junior,SCS,Direct Admit,4065.0,0.0,0,0,0,0
1,Fall,4068,Fall 2006,F,19,2.700,COLL,COLL,COLL1,CMCLBA,...,N,Sophomore,COLL,Direct Admit,4065.0,0.0,0,0,1,1
2,Fall,4068,Fall 2006,F,18,0.000,BUS,BUS,BUS1,BUSBSB,...,N,Freshman,BUS,Direct Admit,4068.0,0.0,0,0,1,1
3,Fall,4068,Fall 2006,F,18,0.000,BUS,BUS,BUS1,MARKBSB,...,N,Freshman,BUS,Direct Admit,4068.0,0.0,0,0,0,1
4,Fall,4068,Fall 2006,F,18,0.000,BUS,BUS,BUS1,BEPPPBSB,...,N,Freshman,BUS,Direct Admit,4068.0,0.0,1,1,0,1
5,Fall,4068,Fall 2006,F,18,0.000,BUS,BUS,BUS1,ENTREBSB,...,N,Freshman,BUS,Direct Admit,4068.0,0.0,0,0,0,1
6,Fall,4068,Fall 2006,F,18,0.000,COLL,COLL,COLL1,NELCBA,...,N,Freshman,COLL,Direct Admit,4068.0,0.0,0,0,1,1
7,Spring,4082,Spring 2008,P,21,2.126,SCS,SCS,SCS1,GENSTAAGS,...,N,Associate,SCS,Direct Admit,4068.0,0.0,0,0,0,1
8,Fall,4068,Fall 2006,F,20,2.470,COLL,COLL,COLL1,FINASBA,...,N,Sophomore,COLL,Direct Admit,4068.0,0.0,0,0,1,1
9,Fall,4068,Fall 2006,F,18,0.000,BUS,BUS,BUS1,FINBSB,...,N,Freshman,BUS,Direct Admit,4068.0,0.0,0,0,1,1


In [4]:
model = SkipGramModel(stem_data.embedding_id, stem_data.train_students, stem_data.valid_students)

Model: "Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Course_ids (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Context (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
Course_embedding (Embedding)    (None, 1, 218)       1583770     Course_ids[0][0]                 
__________________________________________________________________________________________________
Context_embedding (Embedding)   (None, 1, 218)       1583770     Context[0][0]                    
______________________________________________________________________________________________

In [5]:
#model.train(stem_data)

In [6]:
model.load('/Users/rsciagli/documents/Fall2020/BAR/skip_gram_weights/skip_gram_model01.hdf5')

In [ ]:
## the inference here is there is a 27% chance course 5 is taken with course 10, and a 24% chance course 6 is 
## taken with course 6
#### is it worth plotting  ?  how can we generalize this a little to visualize it.
model.model.predict([np.array([[5],[6]]),np.array([[10],[6]])])
# for i in stem_data.train_generator():
#     print(i)

In [8]:
## pulling out the embeddings to prep for comparing vectors using only the embeddings get_embeddings() gives 
## back embedding matrix
#model_embeddings = model.get_embedding()

stem_data.set_embedding(model.get_embedding())

In [ ]:
def demographic_valid_generator(self):
    while True:
        for (student, term), df in self.crs_df_valid.groupby(['PRSN_UNIV_ID','ACAD_TERM_CD']):
            courses_set = self.make_set(df)
            if len(courses_set) > 1:
                # calculate the vector describing courses
                x = self.embedding[list(courses_set)] / len(courses_set)

                # gather demographic info
                row = self.student_attr_df[self.student_attr_df['PRSN_UNIV_ID']==student]
                ethnicity = row['ethnicity_id'].values[0] ## and grab row from student attributes that matches row for student var
                urm_flag = row['urm_flag_id'].values[0]
                gender = row['gender_id'].values[0]
                pell_status = row['pell_eligibility_id'].values[0]

                y = np.array([ethnicity, urm_flag, gender, pell_status])
                # yield data
                yield x,y

stem_data.demographic_valid_generator = demographic_valid_generator

In [ ]:
for x,y in demographic_valid_generator(stem_data):
    print(y)

In [ ]:
## this takes in a course id and finds its associated course name
def course_to_label(course):
    return stem_data.crs_df[stem_data.crs_df['agg_id']==course].iloc[0]['CRS_NM']

In [ ]:
course_to_label(stem_data.embedding_id[210])

In [ ]:
dept_with_orphans = stem_data.crs_df[stem_data.crs_df['agg_id']==stem_data.crs_df['CRS_SUBJ_DEPT_CD']]['agg_id'].unique()
stem_data.course_to_id['other_dept']

In [ ]:
## argsort gives us courses that are similar in the order of least like to most like of all the embedding ids
np.argsort(model_embeddings[3]@model_embeddings.T)
print(course_to_label(stem_data.embedding_id[4]))
print(course_to_label(stem_data.embedding_id[123]))

In [ ]:
## this gives the cosine similarity (dot) for two courses.  the larger the number the higher the correlation,
## negative value means anticorrelated. zero means no correlation.
## we will need to normalize the vectors at some point.
model_embeddings[3]@model_embeddings[123]

In [ ]:
def course_cosine_similarity(course1, course2):
    crs1 = course_to_label(stem_data.embedding_id[course1])
    crs2 = course_to_label(stem_data.embedding_id[course2])
    vec_1 = model_embeddings[course1]
    vec_2 = model_embeddings[course2]
    correlation = np.dot(vec_1, vec_2)/np.linalg.norm(vec_1)/np.linalg.norm(vec_2)
    
    return crs1, crs2, correlation     

In [ ]:
course_cosine_similarity(136,3026)

In [ ]:
def course_correlation(crs):
    correlation = np.argsort(model_embeddings[crs]@model_embeddings.T)
    #name = course_to_label(stem_data.embedding_id[course1])
    vec_1 = model_embeddings[crs]
    correlation = np.dot(model_embeddings, vec_1)/np.linalg.norm(vec_1)/np.linalg.norm(model_embeddings, axis=1)
    #return list(sorted(correlation))[-20:]
    highest_matches = np.argsort(correlation)[-10:]
    for i in highest_matches:
        print(i, course_to_label(stem_data.embedding_id[i]), correlation[i])
        #return(vec_1, i, course_to_label(stem_data.embedding_id[i]), correlation[i])
        
    lowest_matches = np.argsort(correlation)[:10]
    for i in lowest_matches:
        print(i, course_to_label(stem_data.embedding_id[i]), correlation[i])
        #return(vec_1, i, course_to_label(stem_data.embedding_id[i]), correlation[i])

In [ ]:
#normalized_embeddings = model_embeddings/np.linalg.norm(model_embeddings, axis=1)

In [ ]:
name_to_course = dict(stem_data.crs_df[['CRS_NM','agg_id']].values)

In [ ]:
stem_data.course_to_id[name_to_course['CSCI-C311']]
                                      

In [ ]:
course_cosine_similarity(3225,5511)

In [ ]:
dif_a = model_embeddings[1122]-model_embeddings[2188]
dif_b = model_embeddings[1617]-model_embeddings[85]
#dif_c = model_embeddings[84]-model_embeddings[869]

In [ ]:
np.dot(dif_a, dif_b)/np.linalg.norm(dif_a)/np.linalg.norm(dif_b)

In [ ]:
#temp = course_correlation(3026)
#temp1 = pd.DataFrame(course_correlation(3026))

In [ ]:
df_model_embeddings = pd.DataFrame(model_embeddings[[3026,5303,290,3225,5881,280,2188,1195,333,3425]].T,
                                                    columns=['CSCI-C241','CSCI-H241','CSCI-A290','CSCI-B351',
                                                    'CSCI-B365','BUS-F260','FRIT-M200','MATH-M301','MATH-M303',
                                                    'MATH-S311'])                                                                                                                                               

In [ ]:
#df_model_embeddings.corr()
#df_model_embeddings.corr().to_csv('CHEM_BS_FIRST4SEM_UGcurr',index=True, sep='\t')

In [ ]:
#plt.hist(df_model_embeddings.var())


In [ ]:
#kot = corr[corr>=.2]
# sb.set(context='notebook', style='white', palette='deep', font='Avenir', font_scale=1.2, color_codes=True, rc=None)
# plt.figure(figsize=(12,10))
# plt.title('Third Semester Math and Computer Science course cosine similarities')
# sb.heatmap(df_model_embeddings.corr(), annot=True, cmap='Greens')

In [ ]:
student_attr = pd.read_excel('/Users/rsciagli/documents/Fall2020/BAR/student_attribute_table_full.xlsx')

In [ ]:
student_attr.columns

In [ ]:
student_attr_df = pd.DataFrame(student_attr)

In [ ]:
student_attr_df['ENTRY_MAJOR_DESC']

In [ ]:
student_attr_df['ETHNICITY'].isnull().value_counts()

In [ ]:
ethnicity_id = list(student_attr_df['ETHNICITY'].unique())
ethnicity_to_id = dict([(ethnicity, i) for i, ethnicity in enumerate(ethnicity_id)])

In [ ]:
ethnicity_to_id

In [ ]:
urm_flag_id = list(student_attr_df['URM_FLAG'].unique())
urm_flag_to_id = dict([(urm, i) for i, urm in enumerate(urm_flag_id)])

In [ ]:
urm_flag_to_id

In [ ]:
gender_id = list(student_attr_df['GENDER'].unique())
gender_to_id = dict([(gender, i) for i, gender in enumerate(gender_id)])

In [ ]:
gender_to_id

In [ ]:
pell_eligibility_id = list(student_attr_df['PELL_ELIGIBILITY'].unique())
pell_status_to_id = dict([(status, i) for i, status in enumerate(pell_eligibility_id)])

In [ ]:
pell_status_to_id

In [ ]:
major_id = list(student_attr_df['ENTRY_MAJOR_DESC'].unique())
major_to_id = dict([(major, i) for i, major in enumerate(major_id)])

intended_school_id = list(student_attr_df['ENTRY_INTENDED_SCHOOL'].unique())
indended_school_to_id = dict([(pre_major, i) for i, pre_major in enumerate(intended_school_id)])

In [ ]:
student_retention = pd.read_csv('/Users/rsciagli/documents/Fall2020/BAR/STUDENT_RETENTION_TABLE_full.csv')

In [ ]:
stu_retention_df = pd.DataFrame(student_retention)

In [ ]:
stu_retention_df.loc[0]

In [ ]:
list(student_retention.columns)

In [ ]:
student_major = pd.read_csv('/Users/rsciagli/documents/Fall2020/BAR/STU_MAJOR_TABLE_full.csv')
student_major.head()